In [1]:
!pip install torch transformers datasets tokenizers numpy tqdm arxiv




  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 764.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.3/231.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.1/344.1 kB 23.7 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=60

In [2]:
import arxiv

def fetch_arxiv_data(query="machine learning", max_results=1000):
    search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.SubmittedDate)
    papers = [result.summary for result in search.results()]
    return papers

arxiv_data = fetch_arxiv_data()

with open("arxiv_data.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(arxiv_data))



<ipython-input-2-3fb4f5d9a654>:5: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  papers = [result.summary for result in search.results()]


In [4]:
import os

print(os.listdir("tokenizer"))


FileNotFoundError: [Errno 2] No such file or directory: 'tokenizer'

In [5]:
import os
from tokenizers import ByteLevelBPETokenizer

# Create the directory if it doesn't exist
os.makedirs("tokenizer", exist_ok=True)

# Train a Byte-Pair Encoding (BPE) tokenizer
tokenizer = ByteLevelBPETokenizer()

tokenizer.train(["arxiv_data.txt"], vocab_size=50_000, min_frequency=2, special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])

# Save tokenizer
tokenizer.save_model("tokenizer")



['tokenizer/vocab.json', 'tokenizer/merges.txt']

In [6]:
import os

print(os.listdir("tokenizer"))  # Check files inside the tokenizer directory


['merges.txt', 'vocab.json']


In [7]:
import os
from tokenizers import ByteLevelBPETokenizer

# Ensure directory exists
os.makedirs("tokenizer", exist_ok=True)

# Train the tokenizer
tokenizer = ByteLevelBPETokenizer()

tokenizer.train(
    files=["arxiv_data.txt"],  # Ensure this file exists
    vocab_size=50_000,
    min_frequency=2,
    special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"]
)

# Save tokenizer
tokenizer.save_model("tokenizer", "tokenizer")

# Convert and save as a single JSON file
tokenizer_json = tokenizer.to_str()
with open("tokenizer/tokenizer.json", "w", encoding="utf-8") as f:
    f.write(tokenizer_json)

print("Tokenizer saved successfully!")


Tokenizer saved successfully!


In [8]:
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="tokenizer/tokenizer.json",
    bos_token="<s>", eos_token="</s>", unk_token="<unk>",
    pad_token="<pad>", mask_token="<mask>"
)

print("Tokenizer loaded successfully!")


Tokenizer loaded successfully!


In [9]:
# Tokenizing the dataset
def tokenize_data(texts):
    return tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)

tokenized_data = tokenize_data(arxiv_data)

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super().__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embed_size, num_heads=heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attn_output, _ = self.attention(query, key, value, attn_mask=mask)
        x = self.norm1(attn_output + query)
        forward_out = self.feed_forward(x)
        return self.norm2(forward_out + x)

class GPT(nn.Module):
    def __init__(self, vocab_size, embed_size=256, num_layers=8, heads=8, dropout=0.1, forward_expansion=4):
        super().__init__()
        self.embed_size = embed_size
        self.word_embedding = nn.Embedding(vocab_size, embed_size)
        self.position_embedding = nn.Embedding(512, embed_size)
        self.layers = nn.ModuleList([TransformerBlock(embed_size, heads, dropout, forward_expansion) for _ in range(num_layers)])
        self.fc_out = nn.Linear(embed_size, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(x.device)
        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

        for layer in self.layers:
            out = layer(out, out, out, mask)

        return self.fc_out(out)



In [11]:
import torch.optim as optim
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

VOCAB_SIZE = tokenizer.vocab_size
BATCH_SIZE = 32
SEQ_LENGTH = 128
EPOCHS = 5

model = GPT(vocab_size=VOCAB_SIZE).to(device)
optimizer = optim.AdamW(model.parameters(), lr=3e-4)
loss_fn = nn.CrossEntropyLoss()

def train_loop(data, model, optimizer, loss_fn, epochs):
    model.train()

    for epoch in range(epochs):
        loop = tqdm(range(0, len(data), BATCH_SIZE), leave=True)
        for i in loop:
            batch = torch.tensor(data[i: i + BATCH_SIZE], dtype=torch.long).to(device)
            target = batch[:, 1:].contiguous()
            input_data = batch[:, :-1].contiguous()

            optimizer.zero_grad()
            output = model(input_data)

            loss = loss_fn(output.view(-1, VOCAB_SIZE), target.view(-1))
            loss.backward()
            optimizer.step()

            loop.set_description(f"Epoch {epoch+1}/{EPOCHS}")
            loop.set_postfix(loss=loss.item())

train_loop(tokenized_data["input_ids"], model, optimizer, loss_fn, EPOCHS)


  0%|          | 0/32 [00:00<?, ?it/s]<ipython-input-11-3803f141a57a>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(data[i: i + BATCH_SIZE], dtype=torch.long).to(device)
Epoch 5/5: 100%|██████████| 32/32 [07:49<00:00, 14.67s/it, loss=3.19]


In [16]:
import torch

def generate_text(model, tokenizer, start_text, max_len=100):
    model.eval()

    tokens = tokenizer.encode(start_text, return_tensors="pt").to(device)

    for _ in range(max_len):
        with torch.no_grad():
            output = model(tokens)  # Ensure model returns logits
            logits = output.logits if hasattr(output, "logits") else output  # Handle different model outputs
            next_token = torch.argmax(logits[:, -1, :], dim=-1).unsqueeze(0)
            tokens = torch.cat([tokens, next_token], dim=1)

    return tokenizer.decode(tokens.squeeze().detach().cpu().tolist())

# Example usage:
print(generate_text(model, tokenizer, "In this paper, we propose a novel approach to"))


In this paper, we propose a novel approach to


.
,
,
,
.
,
.
.
,
.
.
,
.
.
.
,
.
,
,
,
,
.
,
,
.
.
,
,
,
,
,
,
,
,
.
,
,
,
,
,
,
,
,
.
.
.
,
.
,
